# Object References, Mutability and Recycling

Think about python variables as labels not as boxes. They are the equivalent of Java's references.

In [1]:
a = [1,3,4]
b = a
a.append(5)
b

[1, 3, 4, 5]

Variables are assigned to objects, not the other way around. The right hand of the assignment happens first.

In [ ]:
# Variables are assigned to objects only after the objects are created

class Gigi:
    def __init__(self):
        print("Gigi id: %d" % id(self))

x = Gigi()
y = Gigi()*10


Gigi id: 2944633497520
<generator object <genexpr> at 0x000002AD9B3792F0>
Gigi id: 2944631370320


TypeError: unsupported operand type(s) for *: 'Gigi' and 'int'

In [4]:
# variable y was not created
print(dir())

['Gigi', 'In', 'Out', '_', '_1', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '__vsc_ipynb_file__', '_dh', '_i', '_i1', '_i2', '_i3', '_i4', '_ih', '_ii', '_iii', '_oh', 'a', 'b', 'exit', 'get_ipython', 'open', 'quit', 'x']


## Identity Equality and Aliases

In [ ]:
# charles and lewis refer to the same object (lewis is an alias for charles)

charles = {"name": "charles dogson", "born": 1832}
lewis = charles
print(charles is lewis)
print(id(charles), id(lewis))
lewis["balance"] = 13
print(charles)

True
3103143447360 3103143447360
{'name': 'charles dogson', 'born': 1832, 'balance': 13}


In [3]:
# alex and lewis are the same object but alex is not

alex = {"name": "charles dogson", "born": 1832, "balance": 13}
print(alex == charles)
alex is not charles

True


True

## Tuples may change

Tuples hold references to object, those references can not change, but the object may change if they are mutable.

In [5]:
t1 = (1,2,[20,30])
t2 = (1,2,(20,30))
print(t1 == t2)
print(id(t1[-1]))
t1[-1].append(99)
print(t1)
print(id(t1[-1]))
t1 == t2

False
3103142850752
(1, 2, [20, 30, 99])
3103142850752


False

## Copies of objects
The easiest way to copy an object is to use the built in constructor type. Or the slicing notation `d = c[:]`. <br>
These methods though produce a **shallow copy**. A shallow copy copies all the reference variables inside of the copied object, but not the referenced objects themselves. <br>
Shallow copies are good to save memory but if not all the referenced objects are immutable you could have bad surprises.

In [7]:
l1 = [1,2,3,[4,5],(6,7)]
l2 = list(l1)
print(l2)
print(l2 == l1)
l2 is l1 

[1, 2, 3, [4, 5], (6, 7)]
True


False

In [8]:
# Example making a shallow copy of a nested list

l1 = [3, [66, 55, 44], (7,8,9)]
l2 = list(l1)
l1.append(100)
l1[1].remove(55)
print("l1: ", l1)
print("l2: ", l2)
l2[1] += [33,22]
l2[2] += (10,11)
print("l1: ", l1)
print("l2: ", l2)


l1:  [3, [66, 44], (7, 8, 9), 100]
l2:  [3, [66, 44], (7, 8, 9)]
l1:  [3, [66, 44, 33, 22], (7, 8, 9), 100]
l2:  [3, [66, 44, 33, 22], (7, 8, 9, 10, 11)]


### Deep and Shallow copies of arbitrary objects

In [12]:
class Bus:
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
    
    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)



import copy
bus1 = Bus(["Alice", "Bill", "Claire", "David"])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)

print(id(bus1), id(bus2), id(bus3))
bus1.drop("Bill")
print(bus2.passengers)
print(id(bus1.passengers), id(bus2.passengers), id(bus3.passengers))
bus3.passengers

3103143343984 3103144661136 3103144660496
['Alice', 'Claire', 'David']
3103167933248 3103167933248 3103142980032


['Alice', 'Bill', 'Claire', 'David']

Deepcopy remember the objects already copied to handle cyclic references gracefully

In [13]:
# cyclic references
from copy import deepcopy

a = [10,20]
b= [a,30]
a.append(b)
print(a)
c = deepcopy(a)
print(c)

[10, 20, [[...], 30]]
[10, 20, [[...], 30]]


## Function parameters as references
In python parameters are always passed by reference (also said *call by sharing*). <br>
As a result a function may change any mutable object passed as paremeter, but cannot change their ids (identities).

In [1]:
# A function may change any mutable object it receives

def f(a,b):
    a += b
    return a

x=1
y=2
print(f(x,y))
print(x, y)
a = [1,2]
b = [3,4]
print(f(a,b))
print(a, b)
t = (10,20)
u = (30,40)
print(f(t,u))
print(t, u)

3
1 2
[1, 2, 3, 4]
[1, 2, 3, 4] [3, 4]
(10, 20, 30, 40)
(10, 20) (30, 40)


## Mutable Types as parameter defaults: BAD IDEA

In [3]:
# Simple class to show the danger of a mutable default

class HauntedBus:
    def __init__(self, passengers:list =[]):
        self.passengers = passengers

    def pick(self, name):
        self.passengers.append(name)
    
    def drop(self, name):
        self.passengers.remove(name)


bus1 = HauntedBus(["Alice", "Bill"])
print(bus1.passengers)
bus1.pick("Charlie")
bus1.drop("Alice")
print(bus1.passengers)
bus2 = HauntedBus()
bus2.pick("Gigi")
print(bus2.passengers)
bus3 = HauntedBus()
print(bus3.passengers)
bus3.pick("Dave")
print(bus2.passengers)
print(bus2.passengers is bus3.passengers)
print(HauntedBus.__init__.__defaults__)

['Alice', 'Bill']
['Bill', 'Charlie']
['Gigi']
['Gigi']
['Gigi', 'Dave']
True
(['Gigi', 'Dave'],)


## Defensive programming with mutable parameters

We should follow the *Principle of least astonishment* and avoid to directly modify mutable objects passed as arguments. <br>
The best practice thus is to copy the referenced mutable object inside of the callable before updating it.

In [4]:
# Example of modifying mutable input arguments inside the caller

class TwilightBus:
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = passengers

    def pick(self, name):
        self.passengers.append(name)
    
    def drop(self, name):
        self.passengers.remove(name)

basketball_team = ["Gino", "Pino", "Luigi", "Mario"]
bus = TwilightBus(basketball_team)
bus.drop("Luigi")
bus.drop("Mario")
print(basketball_team)

['Gino', 'Pino']


## del and garbage collection

Objects in Python are never explicitly destroyed; however, when they become unreachaable they may be garbage-collected. <br>
The `del` statement deletes names, not objects.

In [9]:
import weakref

s1 = {1,3,4}
s2 = s1
def bye():
    print('Gone')
ender = weakref.finalize(s1, bye)
print(ender.alive)
del s1
print(ender.alive)
s2 = 'spam'
print(ender.alive)

True
True
Gone
False


## Weak references
Sometimes it is useful to keep a reference to an object without increasing the reference count. This can be useful for example for caching.

In [19]:
import weakref
import time

_ = {0,1}
wref = weakref.ref(_)
print(wref)
print(wref())
_={2,3,4}
print(wref())





<weakref at 0x000001CEBE264720; to 'set' at 0x000001CEBE206340>
{0, 1}
None


## WeakValueDictionary
Commonly used for caching

In [1]:
class Cheese:
    def __init__(self, kind):
        self.kind = kind
    
    def __repr__(self):
        return f"Cheese({self.kind})"
    

import weakref
stock = weakref.WeakValueDictionary()
catalog = [Cheese('RedLeicester'), Cheese("Tilsit"), Cheese("Brie"), Cheese("Parmesan")]
for cheese in catalog:
    stock[cheese.kind] = cheese

print(sorted(stock.keys()))

del catalog
print(sorted(stock.keys()))
del cheese
print(sorted(stock.keys()))

['Brie', 'Parmesan', 'RedLeicester', 'Tilsit']
['Parmesan']
[]


### Limitations of weak references
Wak references can not target all objects, like list and dict. In that case you would need to subclass the object first, in order to refernce it. <br>
int and tuples though cannot be referenced even if subclasses of those types are created.